In [ ]:
import spacy
from collections import Counter
from geopy.geocoders import Nominatim
import folium

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Text containing place names and relations
text = "From Penrith two roads lead to Pooley Bridge, about six miles distant, which spans the Eamont just at its issue from Ulleswater"

# Extract entities from text using spaCy
doc = nlp(text)
entities = [(entity.text, entity.label_) for entity in doc.ents]

# Filter out entities that are not locations
locations = []
for entity in entities:
    if entity[1] in ['GPE', 'PERSON', 'ORG']:
        locations.append(entity[0])

# Get coordinates for locations using GeoPy
geolocator = Nominatim(user_agent="my-application")
coordinates = []
for location in locations:
    location_data = geolocator.geocode(location)
    if location_data is not None:
        coordinates.append((location_data.latitude, location_data.longitude))

# Create map using Folium and add markers for locations
m = folium.Map(location=[coordinates[0][0], coordinates[0][1]], zoom_start=5)
for coord in coordinates:
    folium.Marker(location=[coord[0], coord[1]]).add_to(m)

# Display map
m


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import spacy
from spacy import displacy

# Load the NER model
nlp = spacy.load("en_core_web_sm")

# Define a function to extract entities and their relationships from text
def extract_entities(text):
    doc = nlp(text)
    entities = []
    relations = []

    # Extract named entities
    for ent in doc.ents:
        if ent.label_ == "GPE" or ent.label_ == "LOC":
            entities.append(ent)

    # Extract relationships between named entities
    for chunk in doc.noun_chunks:
        subj = None
        obj = None
        for ent in entities:
            if ent.text == chunk.text:
                if not subj:
                    subj = ent
                else:
                    obj = ent
                    break
        if subj and obj:
            relations.append((subj, obj))

    return entities, relations


# Define a function to disambiguate locations
def disambiguate_locations(entities):
    # Create a set of unique location names
    unique_locs = set()
    for entity in entities:
        unique_locs.add(entity.text)

    # Disambiguate location names
    for loc in unique_locs:
        count = 0
        for entity in entities:
            if entity.text == loc:
                count += 1
                if count > 1:
                    entity._.is_ambiguous = True
                else:
                    entity._.is_ambiguous = False

# Define a function to plot entities and their relationships on a map
def plot_map(entities, relations):
    options = {"ents": ["GPE", "LOC"], "colors": {"GPE": "orange", "LOC": "green"}}
    if entities:
        disambiguate_locations(entities)
        displacy.render(entities, style="ent", jupyter=True, options=options)
    if relations:
        displacy.render(relations, style="dep", jupyter=True, options=options)

# # Extract entities and their relationships from multiple sentences
# sentences = ["I live in New York and work in Boston.", "My friend works in Chicago but also visits Los Angeles."]
# entities, relations = []
# for sentence in sentences:
#     sent_entities, sent_relations = extract_entities(sentence)
#     entities += sent_entities
#     relations += sent_relations

# # Plot entities and their relationships on a map
# plot_map(entities, relations)

# Part 2

In [ ]:
# import spacy
# from spacy import displacy

# # Load the NER model
# nlp = spacy.load("en_core_web_sm")

# # Define a function to extract entities and their relationships from text
# def extract_entities(text):
#     doc = nlp(text)
#     entities = []
#     relations = []

#     # Extract named entities
#     for ent in doc.ents:
#         if ent.label_ == "GPE" or ent.label_ == "LOC":
#             entities.append(ent)

#     # Extract relationships between named entities
#     for chunk in doc.noun_chunks:
#         subj = None
#         obj = None
#         for ent in entities:
#             if ent.text == chunk.text:
#                 if not subj:
#                     subj = ent
#                 else:
#                     obj = ent
#                     break
#         if subj and obj:
#             relations.append((subj, obj))

#     return entities, relations


# # Define a function to disambiguate locations
# def disambiguate_locations(entities):
#     # Create a set of unique location names
#     unique_locs = set()
#     for entity in entities:
#         unique_locs.add(entity.text)

#     # Disambiguate location names
#     for loc in unique_locs:
#         count = 0
#         for entity in entities:
#             if entity.text == loc:
#                 count += 1
#                 if count > 1:
#                     entity._.is_ambiguous = True
#                 else:
#                     entity._.is_ambiguous = False

# # Define a function to plot entities and their relationships on a map
# def plot_map(entities, relations):
#     options = {"ents": ["GPE", "LOC"], "colors": {"GPE": "orange", "LOC": "green"}}
#     if entities:
#         disambiguate_locations(entities)
#         displacy.render(entities, style="ent", jupyter=True, options=options)
#     if relations:
#         displacy.render(relations, style="dep", jupyter=True, options=options)

# # Extract entities and their relationships from multiple sentences
# sentences = ["I live in New York and work in Boston.", "My friend works in Chicago but also visits Los Angeles."]
# entities = []
# relations = []
# for sentence in sentences:
#     sent_entities, sent_relations = extract_entities(sentence)
#     entities += sent_entities
#     relations += sent_relations

# # Plot entities and their relationships on a map
# plot_map(entities, relations)

AttributeError: ignored